In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
rdd = sc.parallelize([(1, "a"), (2, "a"), 
                      (3,"a"), (4, "a"), 
                      (5, "a"), (6, "a"),
                      (1,"a"), (7,"a"), 
                      (7,"a"), (9,"a"), (21, "a")], 4)

rdd2 = rdd.mapValues(lambda x: "b")
print rdd.getNumPartitions()
print rdd.partitioner  # has no partitioner
print sc.defaultParallelism

In [ ]:
rdd.glom().collect()

In [ ]:
rdd2.glom().collect()

In [ ]:
rdd.union(rdd2).getNumPartitions()

In [ ]:
rdd.union(rdd2).glom().collect()

In [ ]:
partitioned_rdd = rdd.partitionBy(20)

In [ ]:
partitioned_rdd.partitioner.partitionFunc # has hashing partitioner

In [ ]:
rdd.map(lambda (x,y):(y,x)).partitionBy(20).glom().collect()

In [ ]:
print(hash("a") % 20) # spark knows how to shuffle the data by hashing the key

In [ ]:
rdd.partitionBy(4).glom().collect()

In [ ]:
rdd2.partitionBy(4).glom().collect()

In [ ]:
rdd = rdd.partitionBy(4)
rdd2 = rdd2.partitionBy(4)
rdd.join(rdd2).collect()

In [ ]:
rdd.join(rdd2).glom().collect()

In [ ]:
rdd = rdd.partitionBy(5)
rdd2 = rdd2.partitionBy(7)
rdd.join(rdd2).glom().collect()

# mapPartitions

The first parameter is going to be a partitition of a RDD as iterator. While in the map scenario, the first parameter is an item in the RDD.

#### rdd.map(lambda item: f(item))              $\hspace{27 mm}$                --> item
#### rdd.mapPartitions(lambda iterator: f(iterator)) --> iterator
<br>

In [ ]:
rdd.glom().collect()

In [ ]:
def func(partition):
    return (list(partition),)

rdd.mapPartitions(func).collect()

In [ ]:
def func(partition):
    return [len(list(partition))]

num_of_elements = rdd.mapPartitions(func).collect()

print(num_of_elements)
plt.bar(range(len(num_of_elements)), num_of_elements, width=0.8, color="blue")

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# data file included inside the folder
fil = sc.textFile("Path to data.txt file")\
        .flatMap(lambda line: [(word,1) for word in line.split()])

fil = fil.partitionBy(200)

num_of_elements = fil.mapPartitions(func).collect()

plt.bar(range(len(num_of_elements)), num_of_elements, width=0.8, color="blue")

# mapPartitionWithIndex

The first parameter is going to be index of the partitition and the second parameter is partition itself.

#### rdd.mapPartitions(lambda iterator: f(iterator)) $\hspace{28 mm}$  --> iterator
#### rdd.mapPartitionsWithIndex(lambda index, iterator: f(iterator)) --> iterator


In [ ]:
# sampling data by dumping partitions (not recommended)
def func(index, partition, percent):
    mod = 100/percent
    if index%mod == 0:
        return partition
    else:
        return []
    
# sampling items inside the partitions (recommended)
def func(index, partition, percent):
    mod = 100 / percent
    lis = []
    for idx, item in enumerate(partition):
        if idx % mod == 0:
            lis.append(item)
    return lis
    
print fil.count()
print fil.mapPartitionsWithIndex(lambda index, partition: func(index, partition, 10)).count()

# foreachPartition

This methos does something with rdd, but does not return an RDD back. Generally used to write data to databases.

In [ ]:
import MySQLdb

db = MySQLdb.connect("ip_address","user_name","password","database_name" )
cursor = db.cursor()

cursor.execute('INSERT INTO TESTDB.letter_number VALUES ("%s", %d)' % ("a",123))

db.commit()

In [ ]:
def func(partition):
    db = MySQLdb.connect("ip_address","user_name","password","database_name" )
    cursor = db.cursor()
    for item in partition:
        cursor.execute('INSERT INTO TESTDB.letter_number VALUES ("%s", %d)' % (item[0], item[1]))
    db.commit()
        
rdd.foreachPartition(func)